In [1]:
from context import *

# Clifford Circuit (`clifford`)

## Basic Usage

### Constructors

#### Identity Circuit

`identity_circuit(N)` constructs an identity circuit of $N$ qubits. This will be an empty circuit (without any gate).

In [2]:
qst.identity_circuit(5)

CliffordCircuit(
  ||)

If the qubit number is not specified, it will represent a generic identity circuit which could potentially be applied to unspecified number of qubits.

In [3]:
qst.identity_circuit()

CliffordCircuit(
  ||)

#### Random Clifford Circuits

`brickwall_rcc(N, depth)` constructs a brickwall random Clifford circuit of $N$ qubits with depth $d$.
* $N$ must be even.
* periodic boundary condition is assumed.

In [3]:
qst.brickwall_rcc(6, 2)

CliffordCircuit(
  |[1,2][3,4][5,0]|
  |[0,1][2,3][4,5]|)

`onsite_rcc(N)` constructs an on-site (local) random Clifford circuit of $N$ qubits.

In [4]:
qst.onsite_rcc(5)

CliffordCircuit(
  |[0][1][2][3][4]|)

`global_rcc(N)` constructs a global random Clifford circuit of $N$ qubits.

In [5]:
qst.global_rcc(5)

CliffordCircuit(
  |[0,1,2,3,4]|)

#### Specify Gate Arrangement

Construct random Clifford circuit with general gate arrangement.

In [4]:
circ = qst.identity_circuit()

Use `.gate(*qubits)` to add random Clifford gates to the circuit.

In [5]:
circ.gate(0,1)
circ.gate(2,4)
circ.gate(1,4)
circ.gate(0,2)
circ.gate(3,5)
circ.gate(3,4)
circ

CliffordCircuit(
  |[3,4]|
  |[1,4][0,2]|
  |[0,1][2,4][3,5]|)

Gates will automatically arranged into layers. Each new gate added to the circuit will commute through the layers if it is not blocked by the existing gates.

In [6]:
circ.N

6

**Note**: If the number of qubits was not explicitly defined, it will be dynamically infered from the circuit width, as the largest qubit index of all gates + 1.

#### Navigate in the Circuit

`.layers_forward()` and `.layers_backward()` provides two generators to iterate over layers in forward and backward order resepctively.

In [7]:
list(circ.layers_forward())

[|[0,1][2,4][3,5]|, |[1,4][0,2]|, |[3,4]|]

In [8]:
list(circ.layers_backward())

[|[3,4]|, |[1,4][0,2]|, |[0,1][2,4][3,5]|]

`.first_layer` and `.last_layer` points to the first and the last layers.

In [9]:
circ.first_layer

|[0,1][2,4][3,5]|

In [10]:
circ.last_layer

|[3,4]|

Use `.next_layer` and `.prev_layer` to move forward and backward.

In [11]:
circ.first_layer.next_layer, circ.last_layer.prev_layer

(|[1,4][0,2]|, |[1,4][0,2]|)

Locate a gate in the circuit.

In [12]:
circ.first_layer.next_layer.next_layer.gates[0]

[3,4]

### `CliffordCircuit` Methods

#### Circuit Construction

`.gate(*qubits)` append a **random Clifford gate** to the circuit that acts on a set of specified qubits.
* The gate that is first added to the circuit will be first applied on the state (in forward transformation).
* Two gates are independent if they acts on disjoint set of qubits. Independent gates can be applied in the same layer simutaneously.
* If a new gate adding to the circuit is not independent of the existing gates on the top layer, it will be added to a new layer.

In [6]:
qst.identity_circuit(5).gate(1).gate(3,4).gate(2,3,5)

CliffordCircuit(
  |[2,3,5]|
  |[1][3,4]|)

`.take(gate)` takes in a specific `CliffordGate` object `gate` and append it to the circuit. Gate will be automatically arranged in the circuit.

In [4]:
qst.identity_circuit(5).take(qst.clifford_rotation_gate('-IXYII'))

CliffordCircuit(
  |[1,2]|)

`.compose(circ)` composes the circuit with another circuit. When acting on an object, the first circuit will be applied first (in forward transformation).

In [17]:
circ1 = qst.identity_circuit(5).gate(1).gate(3,4).gate(2,3,5)
circ2 = qst.identity_circuit(5).gate(4).gate(4,5).gate(1)
circ1, circ2

(CliffordCircuit(
   |[2,3,5]|
   |[1][3,4]|),
 CliffordCircuit(
   |[4,5]|
   |[4][1]|))

In [18]:
circ1.compose(circ2)

CliffordCircuit(
  |[4,5]|
  |[2,3,5][4][1]|
  |[1][3,4]|)

**Side Effects:** `.gate`, `.take`, `.compose` are all in-place operations, meaning that the circuit who initiate these method will be modified by these methods. 

`.copy()` returns a new copy of the circuit, which can be used before the in-place operations to prevent the original circuit to be .

#### Unitary Transformation

`CliffordCircuit` can be applied to many objects and implement the unitary transformation. The object only need to be a subclass of `PauliList`. It can be either a state (`StablizerStates`), or an operator (`PauliPolynomial`), or a unitary map (`CliffordMap`).

* `.forward(obj)` unitary transform the object forward
* `.backward(obj)` unitary transform the object backward

**Side Effect:** the transformation is implemented on the object in-place, meaning that the object will be modified by the  unitary transformation.

**Randomness:** The **random Clifford circuit** is not a fixed circuit but an **ensemble of circuits**. When applying a random unitary circuit on an object, everytime the transformation will be *different* (random Clifford gates will be sampled on the fly). Because of this reason, random Clifford circuits *can not* be compiled.

Example: create a state and a circuit

In [22]:
rho = qst.stabilizer_state('+IX','-ZI')
circ = qst.identity_circuit(2)
circ.take(qst.clifford_rotation_gate('XX'))
circ.take(qst.clifford_rotation_gate('YI'))
rho, circ

(StabilizerState(
    +IX
    -ZI),
 CliffordCircuit(
   |[0]|
   |[0,1]|))

Map the state forward.

In [23]:
circ.forward(rho)
rho

StabilizerState(
   +IX
   +YX)

Map the state backward.

In [24]:
circ.backward(rho)
rho

StabilizerState(
   +IX
   -ZI)

Unitary mappin is invertible. The original state is restored.

#### Circuit Compilation

`.compile()` compiles the Clifford circuit into a single clifford map, such that the unitary transformation can be implemented more efficiently.

* Before compilation, the circuit forward and backward maps are not specified. The unitary transformation will be performed by acting each layer.

In [25]:
circ.__dict__

{'first_layer': |[0,1]|,
 'last_layer': |[0]|,
 'forward_map': None,
 'backward_map': None,
 'N': 2}

* After compilation, the forward and backward maps are defined. Unitary transformation will be implemented by Clifford map.

In [26]:
circ.compile()
circ.__dict__

{'first_layer': |[0,1]|,
 'last_layer': |[0]|,
 'forward_map': CliffordMap(
   X0-> -ZI
   Z0-> -YX
   X1-> +IX
   Z1-> +ZY),
 'backward_map': CliffordMap(
   X0-> +ZI
   Z0-> +YX
   X1-> +IX
   Z1-> +ZY),
 'N': 2}

#### POVM

`.povm(nsample)` provides a generator to sample $n_\text{sample}$ from the prior POVM based on the circuit by back evolution.

In [3]:
circ = qst.onsite_rcc(3)
list(circ.povm(3))

[StabilizerState(
    -XII
    +IXI
    +IIY),
 StabilizerState(
    -ZII
    -IXI
    +IIZ),
 StabilizerState(
    +YII
    +IXI
    -IIZ)]

## Diagonalization and SBRG

### Diagonalization

#### Diagonalize Pauli Operator

`diagonalize(P, i0=0, afterward=False)` finds a Clifford circuit $R$ to bring transform a Pauli operator $P$ to a diagonal operator acting at qubit $i_0$, i.e.

$$R P R^\dagger = Z_{i_0}.$$

**Parameters:**
* `P` -  the Pauli operator to diagonalize (as a `Pauli` object).
* `i0` - the target qubit position
* `causal` - whether to preserve the causal structure by restricting the action of Clifford transformation to the qubits at i0 and afterwards.

Example: default behavior

In [2]:
P = qst.pauli('YY')
R = qst.diagonalize(P).compile()
print('forward map:\n{}\ntransformed operator:\n{}'.format(R.forward_map, R.forward(P)))

forward map:
CliffordMap(
  X0-> +XI
  Z0-> -YY
  X1-> -XZ
  Z1-> +XX)
transformed operator:
 +ZI


Specify the qubit position

In [3]:
P = qst.pauli('ZZIIX')
qst.diagonalize(P, 2).forward(P)

 +IIZII

In [4]:
P = qst.pauli('ZZIIX')
qst.diagonalize(P, 2, causal=True).forward(P)

 +ZZZII

#### Diagonalize Stabilizer Group

`diagonalize(rho)` finds the Clifford circuit to diagonalize the stabilizer state `rho`, by transforming to a new basis in which the stabilizer group elements are simutaneously diagonalized. 

**Parameters:**
* `rho` - the stabilizer state to be diagonalized (as `StabilizerState` object).

Example: start with a random stabilizer state

In [6]:
rho = qst.random_clifford_state(5)
rho

StabilizerState(
   -XYXIZ
   +IZIYX
   +ZXIXI
   +IYZZX
   +YIYZZ)

Find the Clifford circuit to diagonalize the state

In [7]:
R = qst.diagonalize(rho)
R.forward(rho)

StabilizerState(
   +ZIIII
   +IZIII
   +IIZII
   +IIIZI
   +IIIIZ)

The Clifford circuit corresponds the the following map.

In [9]:
R.compile().forward_map

CliffordMap(
  X0-> +IIXIY
  Z0-> -YIIZX
  X1-> +XYIXI
  Z1-> +YIYXZ
  X2-> -ZIZYX
  Z2-> -YZZIX
  X3-> -ZYZYX
  Z3-> +IXXZI
  X4-> -XIIIY
  Z4-> +ZYIYI)

### SBRG

Diagonalize random Hamiltonian by the SBRG algorithm. See https://arxiv.org/abs/1508.03635.

In [15]:
ham = qst.pauli_zero(4)
for i in range(4):
    ham += numpy.random.rand() * qst.pauli({i:'Z',(i+1)%4:'Z'}, 4)
    ham += numpy.random.rand() * qst.pauli({i:'X'}, 4)
ham

0.63 IIIX +0.11 IIZZ +0.68 IIXI +0.63 IZZI +0.17 IXII +0.81 ZIIZ +0.37 ZZII +0.47 XIII

In [17]:
heff, circ = qst.SBRG(ham)
heff, circ

(0.98 IZII -0.10 IZZI -1.19 ZIII +0.59 ZIZI +0.16 ZZIZ,
 CliffordCircuit(
   |[3]|
   |[2,3]|
   |[0][1,2]|
   |[0,3][1,2]|))

In [18]:
circ.forward(ham.copy())

0.63 YIYZ -0.11 IXZI +0.68 IZII +0.63 IXII -0.17 IZXX -0.81 ZIII +0.37 ZIZI -0.47 XIIX

## Algorithm Deatials

### Classes

#### `CliffordGate` Class

`ClifordGate(*qubits)` represents a Clifford gate acting on a set of qubits.

**Parameters**
- `*qubits`: indices of the set of qubits on which the gate acts on.

Example:

In [20]:
gate = qst.circuit.CliffordGate(0,1)
gate.__dict__

{'qubits': (0, 1),
 'n': 2,
 'generator': None,
 'forward_map': None,
 'backward_map': None}

Without specifying either the generator or the forward/backward maps, the gate will be treated as a random gate. Its action on the state will be stocastic (randomly sampled every time).

In [23]:
[gate.forward(qst.zero_state(2)) for _ in range(3)]

[StabilizerState(
    -ZZ
    -ZI),
 StabilizerState(
    +IZ
    -YI),
 StabilizerState(
    -ZY
    +ZI)]

If the `.generator` is specified to a Pauli operator, the gate will implement Clifford rotation generated by the Pauli operator.

In [26]:
gate = qst.circuit.CliffordGate(0,1)
gate.generator = qst.pauli('-XY')
gate.forward(qst.zero_state(2))

StabilizerState(
   +YY
   -XX)

If the `.forward_map` or the `.backward_map` is specified to a Clifford map, the gauge will implement the Clifford transformation according to the map.

In [28]:
gate = qst.circuit.CliffordGate(0,1)
gate.forward_map = qst.clifford_rotation_map('-XY')
gate.forward(qst.zero_state(2))

StabilizerState(
   +YY
   -XX)

In [29]:
gate = qst.circuit.CliffordGate(0,1)
gate.backward_map = qst.clifford_rotation_map('XY')
gate.forward(qst.zero_state(2))

StabilizerState(
   +YY
   -XX)

#### `CliffordLayer` Class

`CliffordLayer(*gates)` represents a layer of Clifford gates. 

**Parameters:**
* `*gates`: quantum gates contained in the layer.

Example:

In [34]:
gate1 = qst.circuit.CliffordGate(0,1)
gate2 = qst.circuit.CliffordGate(3,5)
layer = qst.circuit.CliffordLayer(gate1, gate2)
layer

|[0,1][3,5]|

In [32]:
layer.__dict__

{'gates': [[0,1], [3,5]],
 'prev_layer': None,
 'next_layer': None,
 'forward_map': None,
 'backward_map': None}

The gates in the same layer should not overlap with each other (all gates need to commute). To ensure this, we do not manually add gates to the layer, but using the higher level function `.gate()` provided by `CliffordCircuit` (see later).

#### `CliffordCircuit` Class

`CliffordCircuit()` represents a quantum circuit of random Clifford gates. It takes no parameters, and is initialized to an indentity circuit, equivalent to `identity_circuit()`.

#### Apply Circuit to State

`.forward(state)` and `.backward(state)` applies the circuit to transform the state forward / backward. 
* Each call will sample a new random realization of the random Clifford circuit.
* The transformation will create a new state, the original state remains untouched.

In [6]:
rho = vaeqst.StabilizerState(6, r=0)
rho

StabilizerState(
 +ZIIIII
 +IZIIII
 +IIZIII
 +IIIZII
 +IIIIZI
 +IIIIIZ)

In [7]:
circ.forward(rho)

StabilizerState(
 -XIXIII
 -IYIIII
 +ZYYZXI
 -IIIZIY
 -ZIYIII
 -IIIYYX)

In [8]:
circ.backward(rho)

StabilizerState(
 -ZXIIII
 +IIYIZI
 -IIZIYI
 +XZYIZZ
 +IIIIIZ
 -IIIXIZ)